In [19]:
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_cluster
from linearmodels import PanelOLS
from linearmodels.panel.results import compare

In [20]:
current_dir = Path.cwd()
parent_dir  = current_dir.parent 
data_processed_dir = os.path.join(parent_dir, 'data\\processed')
panel_data = os.path.join(data_processed_dir, 'panel_dataset.csv')

In [21]:
df = pd.read_csv(panel_data)
df.head()

,state_district,scst,stunting_rate,underweight_rate,wasting_rate,mean_haz,mean_waz,mean_whz,mean_age,mean_hhsize,...,prop_urban,pop_weighted,n_obs,state_name,district_name,survey_month,survey_year,post,phase2,months_since_covid
0,andhra pradesh - anantapur,0,0.424420,0.373835,0.142843,-156.453107,-163.093026,-105.822097,25.765874,5.954224,...,0.315953,386.610788,132,andhra pradesh,anantapur,7,2015,0,0,0
1,andhra pradesh - anantapur,1,0.402882,0.442281,0.169819,-169.162711,-185.365409,-127.838352,26.311161,5.083751,...,0.267077,133.414648,46,andhra pradesh,anantapur,7,2015,0,0,0
2,andhra pradesh - chittoor,0,0.300597,0.290890,0.170514,-101.365416,-126.320578,-101.284249,24.976525,6.072965,...,0.358394,382.116796,118,andhra pradesh,chittoor,6,2015,0,0,0
3,andhra pradesh - chittoor,1,0.388943,0.431217,0.192149,-165.868714,-174.765534,-114.718253,25.762278,6.087762,...,0.179219,174.797475,54,andhra pradesh,chittoor,6,2015,0,0,0
4,andhra pradesh - east godavari,0,0.261943,0.227783,0.137352,-109.833649,-121.023186,-84.910702,24.114466,5.067943,...,0.241745,472.713753,118,andhra pradesh,east godavari,5,2015,0,0,0


In [22]:
# Verify panel structure
print("Panel structure:")
print(f"Unique districts: {df['state_district'].nunique()}")
print(f"Total observations: {len(df)}")
print(f"\nObservations by group:")
print(df.groupby(['post', 'phase2', 'scst']).size())

Panel structure:
Unique districts: 573
Total observations: 2275

Observations by group:
post  phase2  scst
0     0       0       374
              1       380
      1       0       192
              1       191
1     0       0       375
              1       380
      1       0       191
              1       192
dtype: int64


In [23]:
# Create summary statistics
summary = df.groupby(['post', 'phase2', 'scst'])[
    ['stunting_rate', 'wasting_rate', 'underweight_rate']
].mean()

summary['n_obs'] = df.groupby(['post', 'phase2', 'scst']).size()

# Reset index to convert to long format
summary_reset = summary.reset_index()

# Create readable labels
summary_reset['Period'] = summary_reset['post'].map({0: 'Pre', 1: 'Post'})
summary_reset['Phase'] = summary_reset['phase2'].map({0: 'Phase 1', 1: 'Phase 2'})
summary_reset['Caste'] = summary_reset['scst'].map({0: 'FC', 1: 'SC/ST'})

# Select and rename columns
summary_final = summary_reset[['Period', 'Phase', 'Caste', 'stunting_rate', 
                                'wasting_rate', 'underweight_rate', 'n_obs']]

summary_final.columns = ['Period', 'Phase', 'Caste', 'Stunting Rate', 
                         'Wasting Rate', 'Underweight Rate', 'N']

# Export to LaTeX
latex_table = summary_final.to_latex(index=False, float_format="%.3f", escape=False)
summary_file = os.path.join(parent_dir, 'results\\tables\\summary_statistics.tex')
with open(summary_file, 'w') as f:
    f.write(latex_table)

print(summary_final)

  Period    Phase  Caste  Stunting Rate  Wasting Rate  Underweight Rate    N
0    Pre  Phase 1     FC       0.327344      0.195533          0.297612  374
1    Pre  Phase 1  SC/ST       0.398037      0.212830          0.364448  380
2    Pre  Phase 2     FC       0.340784      0.212242          0.322656  192
3    Pre  Phase 2  SC/ST       0.412985      0.244157          0.405488  191
4   Post  Phase 1     FC       0.315390      0.193672          0.268477  375
5   Post  Phase 1  SC/ST       0.387980      0.202015          0.324256  380
6   Post  Phase 2     FC       0.294862      0.152188          0.241120  191
7   Post  Phase 2  SC/ST       0.375427      0.174575          0.305199  192


In [24]:
# Prepare data for panel regression
df = df.set_index(['state_district', 'survey_year'])

# Define outcomes
outcomes_main = ['stunting_rate', 'wasting_rate', 'underweight_rate']
outcomes_robust = ['mean_haz', 'mean_waz', 'mean_whz']

# Store results
results_dict_main = {}
results_dict_robust = {}

# Main DDD regression
for outcome in outcomes_main:
    model = PanelOLS.from_formula(
        f'{outcome} ~ scst + post:phase2 + post:scst + '
        'phase2:scst + post:phase2:scst + months_since_covid:scst + '
        'mean_age + mean_hhsize + mean_wealth + prop_literate + prop_urban + '
        'EntityEffects + TimeEffects',
        data=df,
        drop_absorbed=True
    )
    
    results = model.fit(cov_type='clustered', cluster_entity=True)
    results_dict_main[outcome] = results

# Robustness DDD regression
for outcome in outcomes_robust:
    model = PanelOLS.from_formula(
        f'{outcome} ~ scst + post:phase2 + post:scst + '
        'phase2:scst + post:phase2:scst + months_since_covid:scst + '
        'mean_age + mean_hhsize + mean_wealth + prop_literate + prop_urban + '
        'EntityEffects + TimeEffects',
        data=df,
        drop_absorbed=True
    )
    
    results = model.fit(cov_type='clustered', cluster_entity=True)
    results_dict_robust[outcome] = results

In [25]:
# Create comparison table
comparison_main = compare(results_dict_main, stars=True)
comparison_robust = compare(results_dict_robust, stars=True)

# Save to LaTeX
reg_table_main = os.path.join(parent_dir, 'results\\tables\\ddd_regression_table_main.tex')
with open(reg_table_main, 'w') as f:
    f.write(comparison_main.summary.as_latex())

reg_table_robust = os.path.join(parent_dir, 'results\\tables\\ddd_regression_table_robust.tex')
with open(reg_table_robust, 'w') as f:
    f.write(comparison_robust.summary.as_latex())    

In [ ]:
# Placebo Test
outcome = 'prop_male'
model = PanelOLS.from_formula(
        f'{outcome} ~ scst + post:phase2 + post:scst + '
        'phase2:scst + post:phase2:scst + months_since_covid:scst + '
        'mean_age + mean_hhsize + mean_wealth + prop_literate + prop_urban + '
        'EntityEffects + TimeEffects',
        data=df,
        drop_absorbed=True
    )
    
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:              prop_male   R-squared:                        0.0174
Estimator:                   PanelOLS   R-squared (Between):              0.1340
No. Observations:                2275   R-squared (Within):               0.0129
Date:                Mon, Dec 08 2025   R-squared (Overall):              0.1328
Time:                        19:33:47   Log-likelihood                    2855.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2.7224
Entities:                         573   P-value                           0.0017
Avg Obs:                       3.9703   Distribution:                 F(11,1687)
Min Obs:                       2.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             1.1206
                            